线性回归的从零开始实现，模型是**y** = **Xw** + b + θ，θ服从（0,0.0001）的正态分布。**X**是一个1000*2的矩阵，每个样本包含从标准正态分布中采样的2个特征，线性模型参数**W** = [2，-3.4]<sup>T</sup>，b = 4.2

In [2]:
import random
import torch

产生数据

In [3]:
def synthetic_data(w, b, num_examples):
    """ 生成线性回归的模型 """
    X = torch.normal(0, 1, (num_examples, len(w)))  # 返回一个1000*2的tensor，数据服从标准正态分布
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)  # 加θ
    return X, y.reshape((-1, 1))  # 返回X，y的列向量

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

print('features:',features[0],'\nlabel:',labels[0])  # features中的每一行都包含一个二维数据样本，labels中的每一行都包含一维标签值（一个标量）

features: tensor([0.0555, 0.8720]) 
label: tensor([1.3533])


读取数据集
该函数能打乱数据集集中的样本并以小批量方式获取数据（batch_size）

In [4]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))

    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
    
batch_size = 10

for X,y in data_iter(batch_size, features, labels):
        print(X, '\n', y)
        break

tensor([[ 0.1475, -2.3927],
        [-0.5745,  0.9771],
        [ 0.6162, -1.1894],
        [-0.9051,  0.6513],
        [-0.8294, -0.2915],
        [-0.9675,  0.1235],
        [-0.7385, -0.8236],
        [ 0.5880,  0.4909],
        [-0.2561, -0.7928],
        [-0.2013,  0.1174]]) 
 tensor([[12.6255],
        [-0.2694],
        [ 9.4587],
        [ 0.1815],
        [ 3.5284],
        [ 1.8383],
        [ 5.5123],
        [ 3.7175],
        [ 6.3779],
        [ 3.3995]])


初始化模型参数

In [5]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

初始化参数之后，我们的任务是更新这些参数，直到这些参数足够拟合我们的数据。更新的算法为梯度下降算法（gradient descent）

---------------
定义线性回归模型

In [6]:
def linreg(X, w, b):
    """ 线性回归模型 """
    return torch.matmul(X, w) + b

定义损失函数，这里使用平方损失函数

In [ ]:
def squared_loss(y_hat, y):
    """ 均方损失 """
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

定义优化算法，主要使用小批量随机梯度下降。

In [7]:
def sgg(params, lr, batch_size):
    """ 小批量随机梯度下降 """
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size  # 计算的损失是一个批量样本的总和，所以用batch_size来归一化步长
            param.grad.zero_()